# Visualize RNN Prediction

In [1]:
using Flux
using StaticArrays
using ProgressMeter
using POMDPs
using POMDPToolbox
using AutomotiveDrivingModels
using AutoViz
using AutomotivePOMDPs
using AutomotiveSensors
using PedCar
using Reel
using BSON: @load

**environment**

In [31]:
mdp = PedCarMDP(pos_res=2.0, vel_res=2., ped_birth=0.7, car_birth=0.7)
pomdp = UrbanPOMDP(env=mdp.env,
                    sensor = GaussianSensor(false_positive_rate=0.05, 
                                            pos_noise = LinearNoise(min_noise=0.5, increase_rate=0.05), 
                                            vel_noise = LinearNoise(min_noise=0.5, increase_rate=0.05)),
                   ego_goal = LaneTag(2, 1),
                   max_cars=1, 
                   max_peds=1, 
                   car_birth=0.7, 
                   ped_birth=0.7, 
                   obstacles=false, # no fixed obstacles
                   lidar=false,
                   ego_start=20,
                   ΔT=0.1)

rng = MersenneTwister(1);

In [32]:
n_models = 5
models = Vector{Chain}(n_models)
for i=1:n_models
    models[i] = BSON.load("model_$(i)0.bson")[:model] 
end


In [33]:
# input_length = n_dims(pomdp)
# output_length = n_dims(pomdp)
# model = Chain(LSTM(input_length, 128),
#               Dense(128, output_length))

In [34]:
function process_prediction(b::Vector{Float64}, o::Vector{Float64})
    b_ = zeros(length(o))
    b_[1:4] = o[1:4] # ego state fully observable
    b_[5:end] = b
    return b_
end

process_prediction (generic function with 1 method)

In [35]:
n_steps = 400
belief_history = Vector{Vector{Float64}}[]
state_history = Scene[]
obs_history = Vector{Float64}[]
s = initial_state(pomdp, rng)
for m in models
    Flux.reset!(m)
end
@time for i=1:n_steps
    if isterminal(pomdp, s)
        break 
    end
    push!(state_history, s)
    a = UrbanAction(0.)
    o = generate_o(pomdp, s, a, s, rng)
    beliefs = Vector{Vector{Float64}}(n_models)
    for i=1:n_models       
        b = models[i](o).tracker.data
        b_ = process_prediction(b, o)
        beliefs[i] = b_
    end
    push!(belief_history, beliefs)
    push!(obs_history, o)
    sp = generate_s(pomdp, s, a, rng)
    s = sp
end


  2.252001 seconds (3.47 M allocations: 395.752 MiB, 26.89% gc time)


In [ ]:
duration, fps, render_hist = animate_scenes(state_history, obs_history, belief_history, pomdp, sim_dt=pomdp.ΔT)
speed_factor = 1
film = roll(render_hist, fps = speed_factor*fps, duration = duration/speed_factor)

In [7]:
function AutomotivePOMDPs.animate_scenes(scenes::Vector{Scene}, observations::Vector{Vector{Float64}}, beliefs::Vector{Vector{Vector{Float64}}}, pomdp::UrbanPOMDP;
                                         sim_dt = 0.1,
                                         cam = StaticCamera(VecE2(0., -8.), 16.0))
    env = pomdp.env 
    duration = length(scenes)*sim_dt
    fps = Int(1/sim_dt)
    function render_rec(t, dt)
        frame_index = Int(floor(t/dt)) + 1
        overlays = SceneOverlay[IDOverlay()]
        obs = [veh for veh in obs_to_scene(pomdp, observations[frame_index]) if veh.id != EGO_ID]
        obs_overlay = GaussianSensorOverlay(sensor=pomdp.sensor, o=obs, color=MONOKAI["color2"])
        push!(overlays, obs_overlay)
        for b in beliefs[frame_index]
            bel = [veh for veh in obs_to_scene(pomdp, b) if veh.id != EGO_ID]
            bel_overlay = GaussianSensorOverlay(sensor=pomdp.sensor, o=bel, color=MONOKAI["color1"])            
            push!(overlays, bel_overlay)
        end
        return AutoViz.render(scenes[frame_index], env, overlays, cam=cam)
    end
    return duration, fps, render_rec
end
    
    